In [1]:
import sys, os, time
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%matplotlib tk
import numpy as np
import select
import json
import yaml
import matplotlib.pyplot as plt
from collections import namedtuple, OrderedDict
import multiprocessing as mp
import random
from string import Formatter
from itertools import chain, product
# import gsm
import gsm
from gsm import tdict, tlist, tset, tstack, tdeque
from gsm import GameObject, GameLogger
from gsm.mixins import Named
from gsm import Array
from gsm import viz
from gsm import util
from examples.tictactoe import TicTacToe
from gsm.common.world import grid
np.set_printoptions(linewidth=120)

/mnt/c/Users/anwan/Nextcloud/Khan/projects/gsm/git/gsm/io/registry.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  info = yaml.load(open(path, 'r'))


In [2]:
seed = 0
# seed = None
play = False
I = viz.Ipython_Runner(r'http://localhost:5000/', 'me', full_log=True, god_mode=True, seed=seed)

In [3]:
I.restart(debug=True)

'Host restarted (debug=True)'

In [4]:
I.available_games()

['catan', 'ttt']

In [5]:
I.select_game('catan')

'Game set to: catan'

In [6]:
I.game_info()

{'name': 'Catan',
 'long_name': 'The Settlers of Catan',
 'short_name': 'catan',
 'num_players': [3, 4],
 'player_names': ['White', 'Red', 'Blue', 'Orange']}

In [7]:
I.game_players()

['White', 'Red', 'Blue', 'Orange']

In [8]:
# I.add_client('randy', 'pal', 'polly', interface='agent', agent_type='random', timeout=None, seed=I.seed)
I.add_client('randy', 'polly', interface='agent', agent_type='pass', timeout=None, seed=I.seed, prob=.5)
I.add_client('pal', interface='agent', agent_type='regular', timeout=None, seed=I.seed)

'Created an interface (agent) for: pal'

In [9]:
I.ping()

{'randy': ['ping reply from pass agent/s: randy, polly', 1.2341830730438232],
 'polly': ['ping reply from pass agent/s: randy, polly', 0.0],
 'pal': ['ping reply from regular agent/s: pal', 1.5717051029205322]}

In [10]:
if play:
    I.add_player('me', 'White')
else:
    I.add_spectator('me')
    I.add_player('pal', 'White')
I.add_player('randy', 'Blue')
I.add_player('polly', 'Red')
# I.add_advisor('pal', 'White')

'polly is now playing Red'

In [11]:
roles = I.get_roles()
if 'me' not in roles:
    I.toggle_pause()
    print(I.toggle_pause())
else:
    print(I.toggle_pause())
roles

auto pause is deactivated


{'pal': 'White', 'polly': 'Red', 'randy': 'Blue'}

In [12]:
# I.begin()
I.load('catan_setup', load_interface=False)

'Game catan_setup.gsm loaded.'

In [13]:
I.cheat('next7')

'Cheat code: next7'

In [14]:
I.status()
I.view()

Received table: 159 entries
-------------
Log
-------------
PLYR:White builds a OBJ[148]:settlement[148] (gaining 1 victory point)
PLYR:White builds a OBJ[149]:road[149]
PLYR:Red builds a OBJ[150]:settlement[150] (gaining 1 victory point)
PLYR:Red builds a OBJ[151]:road[151]
PLYR:Blue builds a OBJ[152]:settlement[152] (gaining 1 victory point)
PLYR:Blue builds a OBJ[153]:road[153]
PLYR:Blue builds a OBJ[154]:settlement[154] (gaining 1 victory point)
PLYR:Blue gains: ore, ore, and sheep
PLYR:Blue builds a OBJ[155]:road[155]
PLYR:Red builds a OBJ[156]:settlement[156] (gaining 1 victory point)
PLYR:Red gains: wood, brick, and ore
PLYR:Red builds a OBJ[157]:road[157]
PLYR:White builds a OBJ[158]:settlement[158] (gaining 1 victory point)
PLYR:White gains: wood, wheat, and wood
Cheat code activated: next7
-*- The next roll will be a 7

Phase: setup
No status found
Active: White


In [16]:
if I.actions is not None:
    I.step()
else:
    I.go()
I.status()
I.view()

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [16]:
# I.save('catan_setup')

In [28]:
viz.render_dict(I.msg)

In [18]:
msg = I.msg
util.obj_cross_ref(msg, {'_obj':msg.table, '_player':msg.players})
for ID, obj in msg.table.items():
    obj._id = ID
msg.keys()

odict_keys(['options', 'status', 'key', 'players', 'table', 'phase', 'log'])

In [19]:
msg.table['10'].neighbors[0].keys()

odict_keys(['row', 'col', 'neighbors', 'obj_type', 'visible', 'edges', 'corners', 'res', 'num', '_id'])

In [31]:
actions = viz.decode_action_set(I.msg.options.loc.actions)
hexs = tlist(I.msg.table[a.ID] for a, in actions)
len(hexs)

18

In [32]:
hexs[0].keys()

odict_keys(['row', 'col', 'neighbors', 'obj_type', 'visible', 'edges', 'corners', 'res', 'num', '_id'])

In [33]:
remaining = tlist()
for h in hexs:
    for c in h.corners:
        if 'building' in c and c.building.player.name != 'White' and c.building.player.num_res > 0:
            remaining.append(h)
            break
hexs = remaining
len(hexs)

8

In [34]:
hexs = tdict({h._id:h for h in hexs})
options = tdict({k:tdict() for k in hexs})
for ID, h in hexs.items():
    options[ID].val = 5 - abs(h.num - 7)
    pass

In [35]:
options

tdict('5':tdict('val':1), '8':tdict('val':2), '9':tdict('val':2), '10':tdict('val':4), '13':tdict('val':3), '14':tdict('val':2), '15':tdict('val':1), '18':tdict('val':3))

In [38]:
hexs['10'].res, hexs['10'].num

('ore', 6)

In [24]:
log = I.get_log()
len(log)

28

In [21]:
for line in log:
    if 'targets' in line:
        print(line['targets'])

[tdict(_player)]
[tdict(_player)]
[tdict(_player)]
[tdict(_player)]


In [29]:
for h in hexs:
    if 'buildings' in h:
        print(h.corners)

In [31]:
hexs[0].corners

[tdict('_obj':'91'), tdict('_obj':'92'), tdict('_obj':'93'), tdict('_obj':'94'), tdict('_obj':'95'), tdict('_obj':'96')]

In [22]:
hexs = tlist()
for ID, obj in I.msg.table.items():
    if obj.obj_type == 'hex':
        hexs.append(obj)
len(hexs)

19